In [2]:
import cv2
from ikomia.dataprocess.workflow import Workflow
from collections import defaultdict
import csv
import pandas as pd

# Задача 1

На основе алгоритма DeepSORT реализуйте возможность подсчета количества объектов и какого класса пересекает заданную линию в обеих направлениях.

In [6]:
input_video_path = 'data/Cars_And_Pedestrians_On_City_Streets_Moscow_preview_2407077.mp4'
output_video_path = 'results/moscow_crossings.avi'

line_pt1 = (1920 // 2, 0)
line_pt2 = (1920 // 2, 1080)

def side_of_line(pt1, pt2, x, y):
    return (pt2[0] - pt1[0]) * (y - pt1[1]) - (pt2[1] - pt1[1]) * (x - pt1[0])

wf = Workflow()
detector = wf.add_task(name='infer_yolo_v9', auto_connect=True)
detector.set_parameters({'model_name': 'yolov9-e'})

tracking = wf.add_task(name='infer_deepsort', auto_connect=True)
tracking.set_parameters({
    'categories': 'all',
    'conf_thres': '0.5',
    'max_age': '50',
    'min_hits': '3',
    'iou_threshold': '0.2',
    'cosine_threshold': '0.2',
    'nn_budget': '100',
    'use_cuda': 'True'
})

stream = cv2.VideoCapture(input_video_path)
if not stream.isOpened():
    print('Error: Could not open video.')
    exit()

w = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = stream.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

prev_side = {}  
counts = defaultdict(lambda: {'to_positive': 0, 'to_negative': 0})

while True:
    ret, frame = stream.read()
    if not ret:
        break

    wf.run_on(array=frame)
    image_out = tracking.get_output(0)
    det_out = tracking.get_output(1)
    
    img_out = image_out.get_image_with_graphics(det_out)    
    objects = det_out.get_objects()  

    for obj in objects:
        track_id = obj.id
        class_name = obj.label               
        x_min, y_min, w_box, h_box = obj.box  
        cx = x_min + w_box / 2
        cy = y_min + h_box / 2

        curr = side_of_line(line_pt1, line_pt2, cx, cy)
        prev = prev_side.get(track_id)

        if prev is not None and curr * prev < 0:
            if curr > 0:
                counts[class_name]['to_positive'] += 1
            else:
                counts[class_name]['to_negative'] += 1

        prev_side[track_id] = curr

    cv2.line(img_out, line_pt1, line_pt2, (0, 255, 0), 2)
    out.write(img_out)
    
stream.release()
out.release()

print("Пересечения по классам")
for cls, dirs in counts.items():
    print(f"{cls}: налево: {dirs['to_positive']} пересечений, направо: {dirs['to_negative']} пересечений")

Will run on cuda


Fusing layers... 
Model summary: 1475 layers, 69529664 parameters, 0 gradients, 245.5 GFLOPs


Successfully loaded imagenet pretrained weights from "/home/danya/Ikomia/Plugins/Python/infer_deepsort/models/checkpoints/osnet_x1_0_imagenet.pth"
Workflow Untitled run successfully in 664.1313630000001 ms.
Workflow Untitled run successfully in 120.62653 ms.
Workflow Untitled run successfully in 122.35372199999999 ms.
Workflow Untitled run successfully in 122.056843 ms.
Workflow Untitled run successfully in 119.022307 ms.
Workflow Untitled run successfully in 120.009073 ms.
Workflow Untitled run successfully in 120.55813900000001 ms.
Workflow Untitled run successfully in 119.352355 ms.
Workflow Untitled run successfully in 119.854873 ms.
Workflow Untitled run successfully in 120.180502 ms.
Workflow Untitled run successfully in 120.926607 ms.
Workflow Untitled run successfully in 119.587334 ms.
Workflow Untitled run successfully in 119.721333 ms.
Workflow Untitled run successfully in 120.188542 ms.
Workflow Untitled run successfully in 122.520221 ms.
Workflow Untitled run successfully i

# Задача 2

На основе алгоритма DeepSORT реализуйте сохранение метаданных трекинга в файл (id, координаты, класс)

In [8]:
input_video_path = 'data/Cars_And_Pedestrians_On_City_Streets_Moscow_preview_2407077.mp4'
output_video_path = 'results/moscow.avi'
metadata_path = 'results/tracking_metadata.csv'

wf = Workflow()
detector = wf.add_task(name='infer_yolo_v9', auto_connect=True)
detector.set_parameters({'model_name': 'yolov9-e'})

tracking = wf.add_task(name='infer_deepsort', auto_connect=True)
tracking.set_parameters({
    'categories': 'all',
    'conf_thres': '0.5',
    'max_age': '50',
    'min_hits': '3',
    'iou_threshold': '0.2',
    'cosine_threshold': '0.2',
    'nn_budget': '100',
    'use_cuda': 'True'
})

cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print('Error: Could not open video.')
    exit()

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

with open(metadata_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    writer.writerow(['frame', 'track_id', 'class', 'x_min', 'y_min', 'width', 'height'])

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        wf.run_on(array=frame)
        image_out = tracking.get_output(0)
        det_out = tracking.get_output(1)

        img_out = image_out.get_image_with_graphics(det_out)
        out.write(img_out)
        
        objects = det_out.get_objects()
        for obj in objects:
            track_id = obj.id
            class_name = obj.label
            x_min, y_min, w_box, h_box = obj.box  
            
            writer.writerow([frame_idx, track_id, class_name, x_min, y_min, w_box, h_box])
        frame_idx += 1

cap.release()
out.release()

Will run on cuda


Fusing layers... 
Model summary: 1475 layers, 69529664 parameters, 0 gradients, 245.5 GFLOPs


Successfully loaded imagenet pretrained weights from "/home/danya/Ikomia/Plugins/Python/infer_deepsort/models/checkpoints/osnet_x1_0_imagenet.pth"
Workflow Untitled run successfully in 657.808622 ms.
Workflow Untitled run successfully in 120.242617 ms.
Workflow Untitled run successfully in 119.528602 ms.
Workflow Untitled run successfully in 118.682777 ms.
Workflow Untitled run successfully in 120.341187 ms.
Workflow Untitled run successfully in 120.970383 ms.
Workflow Untitled run successfully in 120.925873 ms.
Workflow Untitled run successfully in 119.885619 ms.
Workflow Untitled run successfully in 119.470893 ms.
Workflow Untitled run successfully in 121.05850199999999 ms.
Workflow Untitled run successfully in 121.812517 ms.
Workflow Untitled run successfully in 122.186294 ms.
Workflow Untitled run successfully in 122.252674 ms.
Workflow Untitled run successfully in 121.768107 ms.
Workflow Untitled run successfully in 122.158835 ms.
Workflow Untitled run successfully in 123.140158 m

In [10]:
pd.read_csv('results/tracking_metadata.csv')

,frame,track_id,class,x_min,y_min,width,height
0,2,1,person,336.0,665.0,22.0,52.0
1,2,2,car,1482.0,673.0,87.0,37.0
2,2,3,car,970.0,670.0,58.0,30.0
3,2,4,car,1635.0,663.0,253.0,111.0
4,2,5,car,215.0,669.0,106.0,70.0
...,...,...,...,...,...,...,...
1801,256,96,bus,1353.0,632.0,268.0,73.0
1802,256,104,bus,584.0,618.0,230.0,99.0
1803,256,108,car,1564.0,684.0,244.0,94.0
1804,256,110,car,1172.0,671.0,145.0,63.0


# Задача 3

Выведите время жизни каждого трека. То есть отслеживаете отдельный id и считаете его время жизни.

In [3]:
input_video_path = 'data/Cars_And_Pedestrians_On_City_Streets_Moscow_preview_2407077.mp4'
output_video_path = 'results/moscow.avi'
metadata_path = 'results/track_lifetimes.csv'

wf = Workflow()
detector = wf.add_task(name='infer_yolo_v9', auto_connect=True)
detector.set_parameters({'model_name': 'yolov9-e'})

tracking = wf.add_task(name='infer_deepsort', auto_connect=True)
tracking.set_parameters({
    'categories': 'all',
    'conf_thres': '0.5',
    'max_age': '50',
    'min_hits': '3',
    'iou_threshold': '0.2',
    'cosine_threshold': '0.2',
    'nn_budget': '100',
    'use_cuda': 'True'
})

cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print('Error: Could not open video.')
    exit()

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

track_frames = {}

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    wf.run_on(array=frame)
    image_out = tracking.get_output(0)
    det_out = tracking.get_output(1)

    img_out = image_out.get_image_with_graphics(det_out)
    out.write(img_out)

    for obj in det_out.get_objects():
        tid = obj.id
        if tid not in track_frames:
            track_frames[tid] = [frame_idx, frame_idx]
        else:
            track_frames[tid][1] = frame_idx

    frame_idx += 1

cap.release()
out.release()

with open(metadata_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['track_id', 'start_frame', 'end_frame', 'lifetime_frames', 'lifetime_seconds'])

    for tid, (start, end) in sorted(track_frames.items()):
        lifetime_frames = end - start + 1
        lifetime_seconds = lifetime_frames / fps
        writer.writerow([tid, start, end, lifetime_frames, f"{lifetime_seconds:.3f}"])

/home/danya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Will run on cuda


Fusing layers... 
Model summary: 1475 layers, 69529664 parameters, 0 gradients, 245.5 GFLOPs


Successfully loaded imagenet pretrained weights from "/home/danya/Ikomia/Plugins/Python/infer_deepsort/models/checkpoints/osnet_x1_0_imagenet.pth"
Workflow Untitled run successfully in 3123.114454 ms.
Workflow Untitled run successfully in 132.738572 ms.
Workflow Untitled run successfully in 120.355851 ms.
Workflow Untitled run successfully in 122.520102 ms.
Workflow Untitled run successfully in 117.933862 ms.
Workflow Untitled run successfully in 118.790984 ms.
Workflow Untitled run successfully in 117.895512 ms.
Workflow Untitled run successfully in 119.057322 ms.
Workflow Untitled run successfully in 118.89474299999999 ms.
Workflow Untitled run successfully in 117.783783 ms.
Workflow Untitled run successfully in 117.399036 ms.
Workflow Untitled run successfully in 117.89269300000001 ms.
Workflow Untitled run successfully in 118.017632 ms.
Workflow Untitled run successfully in 117.862023 ms.
Workflow Untitled run successfully in 118.731315 ms.
Workflow Untitled run successfully in 121

In [4]:
pd.read_csv('results/track_lifetimes.csv')

,track_id,start_frame,end_frame,lifetime_frames,lifetime_seconds
0,1,2,3,2,0.08
1,2,2,132,131,5.24
2,3,2,16,15,0.60
3,4,2,7,6,0.24
4,5,2,256,255,10.20
...,...,...,...,...,...
69,108,220,256,37,1.48
70,110,227,256,30,1.20
71,114,239,255,17,0.68
72,118,248,249,2,0.08


In [5]:
pd.read_csv('results/track_lifetimes.csv')['lifetime_seconds'].mean()

1.2924324324324323